In [1]:
import pandas as pd
import sumolib
import geopandas as gpd
import xmltodict
import json
from tqdm import tqdm


c:\Users\MehdiMoghadam\anaconda3\lib\site-packages\geopandas\_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()
C:\Users\MehdiMoghadam\AppData\Local\Temp\ipykernel_13860\2684424938.py:4: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


In [4]:


def emission_to_csv(emission_file: str, net_file: str, output_file:str) -> None:
    
    '''
    
    To Convert XML to Geojson or CSV
    
    emission_file: path to XML file
    net_file: path to network file which XML created from (this is to extract the coordinates)
    output_file: path to output
    
    '''
    
    
    with open(emission_file) as xml_file:

        data_dict = xmltodict.parse(xml_file.read())

    json_data = json.dumps(data_dict)
    with open("emission1.json", "w") as json_file:
        json_file.write(json_data)
        
        
    f = open('emission1.json')
    data = json.load(f)    
    
    df = pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]],
                      columns=['@time', '@id',
                               '@eclass','@CO2',
                               '@CO','@HC',
                               '@NOx','@PMx',
                               '@fuel','@electricity',
                               '@noise','@route',
                               '@type','@waiting',
                               '@lane','@pos',
                               '@speed','@angle',
                               '@x','@y'])
    
    
    l =  []
    for i in tqdm(range(len(data['emission-export']['timestep']))):

        time = data['emission-export']['timestep'][i]['@time']
        try:
            if len(data['emission-export']['timestep'][i]) > 1:
                for k in data['emission-export']['timestep'][i]['vehicle']:
                    k['@time'] = time
                    l.append(k)
            else:
                print(i)
                k = data['emission-export']['timestep'][i]['vehicle']
                k['@time'] = time
                l.append(k)
                pass
        except:
            pass


    df = pd.DataFrame(l)
    df = df[[
        '@time', '@id',
        '@eclass','@CO2',
        '@CO','@HC',
        '@NOx','@PMx',
        '@fuel','@electricity',
        '@noise','@route',
        '@type','@waiting',
        '@lane','@pos',
        '@speed','@angle',
        '@x','@y']]


    net = sumolib.net.readNet(net_file)

    df['lat'] = 0
    df['lon'] = 0 
    
    
    for i in tqdm(range(len(df))):

        lon, lat = net.convertXY2LonLat(float(df['@x'][i]), float(df['@y'][i]))
        df['lat'][i], df['lon'][i] = lat, lon


    emission = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
    )

    emission = emission[['lat','lon',
                         'geometry','@time',
                         '@id','@eclass',
                         '@CO2','@CO',
                         '@HC','@NOx',
                         '@PMx','@fuel',
                         '@electricity','@noise',
                         '@route','@type',
                         '@waiting','@lane',
                         '@pos','@speed','@angle']]
    
    print(emission.head())
    emission['date'] = pd.to_datetime(emission['@time'], unit='s')
    emission = emission.iloc[::5]   # selects 1 out of 5 records to make file size smaller
    emission['date'] = emission['date'].astype(str)
    gdf = gpd.GeoDataFrame(
    emission, geometry=emission.geometry, crs="EPSG:4326"
)
    # emission = emission.set_index('date')
    # emission.to_csv(r'C:\Users\MehdiMoghadam\OneDrive - Agilysis Limited\Desktop\Data Collector\Sumo Map\Dashboard Ai Confrence\emission files\Oxf.csv')
    gdf.to_file(output_file, driver='GeoJSON')
    print('Done')
        
    

In [5]:
emission_to_csv(r'C:\Users\MehdiMoghadam\OneDrive - Agilysis Limited\Desktop\Data Collector\Sumo Map\Dashboard Ai Confrence\RL AL confrence\final\rl1.xml',
                r'C:\Users\MehdiMoghadam\OneDrive - Agilysis Limited\Desktop\Data Collector\Sumo Map\Dashboard Ai Confrence\RL AL confrence\final3\Birmingham_new\Birmingham_demo_3_disableturnaround.net.xml',
                r'C:\Users\MehdiMoghadam\OneDrive - Agilysis Limited\Desktop\Data Collector\Sumo Map\Dashboard Ai Confrence\emission files\rl1.geojson')